# NTRK complex modeling

This notebook shows the modeling of tyrosine kinase inhibitors into NTRK1 using functionalities from KinoML from the commit [b252ff6](https://github.com/openkinome/kinoml/commit/b252ff67a35b9149e704d51b7c838dcf1e9fb10f).

In [1]:
import traceback

from kinoml.core.ligands import SmilesLigand
from kinoml.core.components import BaseProtein
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.complexes import OEKLIFSKinaseHybridDockingFeaturizer

In [2]:
tkis = {'larotrectinib': 'O=C(Nc1cnn2ccc(N3CCC[C@@H]3c3cc(F)ccc3F)nc12)N1CC[C@H](O)C1',
        'selitrectinib': 'C[C@@H]1CCc2ncc(F)cc2[C@H]2CCCN2c2ccn3ncc(c3n2)C(=O)N1',
        'repotrectinib': 'C[C@H]1CNC(=O)c2cnn3ccc(nc23)N[C@H](C)c2cc(F)ccc2O1'}
ntrks = {'NTRK1': 480,
         'NTRK2': 481,
         'NTRK3': 482}

In [3]:
docking_featurizer = OEKLIFSKinaseHybridDockingFeaturizer(loop_db="~/.OpenEye/rcsb_spruce.loop_db", shape_overlay=True)

In [4]:
for ntrk, kinase_id in ntrks.items():
    for tki, smiles in tkis.items():
        smiles_ligand = SmilesLigand(smiles=smiles, name=tki)
        base_protein = BaseProtein(name=ntrk)
        base_protein.klifs_kinase_id = kinase_id
        kinase_ligand_complex = ProteinLigandComplex(components=[base_protein, smiles_ligand])
        try:
            system = docking_featurizer.featurize(kinase_ligand_complex)
        except Exception:
            print(f'Error for generating structure of {ntrk} in complex with {tki}!')
            traceback.print_exc()

Error for generating structure of NTRK3 in complex with larotrectinib!


Traceback (most recent call last):
  File "<ipython-input-4-176da7f673a9>", line 8, in <module>
    system = docking_featurizer.featurize(kinase_ligand_complex)
  File "/home/david/miniconda3/envs/kinoml/lib/python3.7/site-packages/kinoml/features/core.py", line 37, in featurize
    features = self._featurize(system)
  File "/home/david/miniconda3/envs/kinoml/lib/python3.7/site-packages/kinoml/features/complexes.py", line 422, in _featurize
    ligand_template.aC_helix,
  File "/home/david/miniconda3/envs/kinoml/lib/python3.7/site-packages/kinoml/features/complexes.py", line 797, in _select_protein_template
    raise NotImplementedError("Would need homology modeling capability.")
NotImplementedError: Would need homology modeling capability.


Error for generating structure of NTRK3 in complex with selitrectinib!


Traceback (most recent call last):
  File "<ipython-input-4-176da7f673a9>", line 8, in <module>
    system = docking_featurizer.featurize(kinase_ligand_complex)
  File "/home/david/miniconda3/envs/kinoml/lib/python3.7/site-packages/kinoml/features/core.py", line 37, in featurize
    features = self._featurize(system)
  File "/home/david/miniconda3/envs/kinoml/lib/python3.7/site-packages/kinoml/features/complexes.py", line 422, in _featurize
    ligand_template.aC_helix,
  File "/home/david/miniconda3/envs/kinoml/lib/python3.7/site-packages/kinoml/features/complexes.py", line 797, in _select_protein_template
    raise NotImplementedError("Would need homology modeling capability.")
NotImplementedError: Would need homology modeling capability.


First, NTRK inhibitor complexes were generated using the default settings (`shape_overlay=False`). In this case, ligand templates, which guide ligand placement during hybrid docking, are searched via fingerprints. This was successful for NTRK1 and NTRK2 in complex with larotrectinib, which used [4yne](https://www.rcsb.org/structure/4YNE) as ligand template. The complexes of NTRK1-3 with selitrectinib show a favorable binding pose. However, the docked ligands clash with the phenylalanine of the DFG motif indicating a wrong kinase conformation. The other complexes were not generated and would require homology modeling.

|kinase|tki          |ligand template|kinase template|kinase conformation|docking|comment|
|------|-------------|---------------|---------------|-------------------|-------|-------|
|NTRK1 |larotrectinib|4yne           |4yne           |DFG in, aC out |+      |                                            |
|NTRK1 |selitrectinib|5jfx           |5jfx           |DFG out, aC out|+/-    |clashing with DFG motif, wrong conformation |
|NTRK1 |repotrectinib|-              |-              |-              |-      |                                            |
|NTRK2 |larotrectinib|4yne           |4at3           |DFG in, aC out |+      |                                            |
|NTRK2 |selitrectinib|5jfx           |4at5           |DFG out, aC out|+/-    |clashing with DFG motif, wrong conformation |
|NTRK2 |repotrectinib|-              |-              |-              |-      |                                            |
|NTRK3 |larotrectinib|-              |-              |-              |-      |                                            |
|NTRK3 |selitrectinib|5jfx           |6kzc           |DFG out, aC out|+/-    |clashing with DFG motif, wrong conformation |
|NTRK3 |repotrectinib|-              |-              |-              |-      |                                            |

Next, complexes were generated with shape_overlay set `True`. Generated complexes of NTRK1 and NTRK2 binding larotrectinib and selitrectinib match the binding mode observed in [Drilon 2017](https://doi.org/10.1158/2159-8290.CD-17-0507) and show a DFG in/aC helix out conformation. Interestingly, the current workflow selects different ligand templates for repotrectinib leading to different binding modes and a different selected kinase conformation. While the complex with NTRK2 is a little clashy, the complex of repotrectinib and NTRK3 looks supprisingly favorable considering the different kinase conformation. Structures of NTRK3 in complex with larotrectinib and selitrectinib could be modeled and would require homology modeling capability.

|kinase|tki          |ligand template|kinase template|kinase conformation|docking|comment                                     |
|------|-------------|---------------|---------------|-|-------|--------------------------------------------|
|NTRK1 |larotrectinib|4yne           |4yne           |DFG in, aC out |+      |            |
|NTRK1 |selitrectinib|4yne           |4yne           |DFG in, aC out |+      |                              |
|NTRK1 |repotrectinib|4yne           |4yne           |DFG in, aC out |+      |                                   |
|NTRK2 |larotrectinib|4yne           |4at3           |DFG in, aC out |+      |                             |
|NTRK2 |selitrectinib|4yne           |4at3           |DFG in, aC out |+      |                                            |
|NTRK2 |repotrectinib|4ymj           |4at5           |DFG out, aC out |+/-   |flipped orientation, clashy, different ligand template selected than for NTRK1|
|NTRK3 |larotrectinib|-             |-             |- |-      |                                            |
|NTRK3 |selitrectinib|-             |-             |- |-      |                                            |
|NTRK3 |repotrectinib|4ymj           |4ymj           |DFG out, aC out |+/-   |flipped orientation, different ligand template selected than for NTRK1|

The NTRK3 structure contains a phospho serine which is not automatically mutated to the standard amino acid. This will be fixed in future releases.